In [ ]:
import pandas as pd
import tqdm
from GetRelationShips import GetRelationShips
from GetEventType import GetEventType
from GetEventDetail import GetEventDetail
from load_model import chatglm
from extractJson import extractJson, JsonValidate
import json
import re

In [2]:
df = pd.read_csv("./qingbao.csv",  on_bad_lines='skip')
res = []
for i in df['origin']:
    res.append(i)
res

['近期，研究人员发现透明部落利用外贸主题的链接进行攻击活动样本。样本为一个伪装成xls的scr文件，会释放持久化组件与RAT对中招用户持续监控。本次攻击活动中使用的RAT既不是其专属木马CrimsonRAT，也不是常用的ObliqueRAT，而是一款简单的RAT，包含屏幕监控、键盘监控、网络传输的功能。通过持久化组件的特征代码相似度判断这是透明部落的攻击活动，在之前的攻击活动中，透明部落使用了一个持久化组件用于驻留，在此次发现的样本中，同样发现了一个相似的持久化组件，二者的代码相似度很高。02疑似Kimsuky组织冒充韩国知名网站Naver发起钓鱼攻',
 '最近，研究人员发现了多个伪装为金融部门正常网站的恶意域，且攻击者似乎从11月初便开始分发冒充Naver Help的钓鱼邮件。其中，NAVER是韩国最大的搜索引擎和门户网站。邮件包含重定向至虚假钓鱼页面的恶意URL，发件人用户名是“Naver Center”，邮件以各种主题来欺骗用户，包括更改联系方式、创建新的一次性密码、从不熟悉的位置登录时邮件存储和阻止访问尝试的通知。一旦用户在钓鱼页面中输入帐户凭据时，将同时转发到攻击者者指定的页面。此外，活动中使用的ne.kr顶级域名与Kimsuky组织经常使用的C2域名格式相同。目前，一些恶意域仍然存在，且某些页面允许用户下载可用于语音钓鱼的恶意APK文件。\xa003CNC组织利用摆渡木马针对军工和教育行',
 'CNC组织最早于2019年被发现，其命名来源于所使用的远控木马的PDB路径信息中包含的cnc_client，且该组织主要针对军工和教育行业进行攻击。近期，研究人员披露了该组织所使用的两个下载器：PrivateImage.png.exe和YodaoCloudMgr.exe。PrivateImage.png.exe下载器具有摆渡攻击的能力，可利用移动存储设备作为“渡船”，间接从隔离网中窃取攻击者感兴趣的文件。即PrivateImage.png.exe会根据是否在%localappdata%路径下选择两种方式执行。若在%localappdata%路径下，其将不断检测是否有新设备接入；若有，则将文件本身复制到新设备中，以便通过可移动设备传播。反之，首先判断%localappdata%\\ImageEditor.exe是否存在：若存在，跳过后续操作并退出。不存在，判断

In [3]:
model = chatglm()

Loading checkpoint shards: 100%|██████████| 7/7 [00:12<00:00,  1.82s/it]


In [4]:
background = """
我希望你能够以一名语言学家的身份帮我完成如下的任务。
首先我会给你一段文本，然后你需要尽可能多的提取出这段文本中发生过的动作事件，比如攻击，渗透，销毁等。
最好一段文本至少能够生成4个以上的词语,词语的长度不能大于6个字。
你的输出需要严格按照python列表的格式输出，我接下来会给你几个例子。
你需要结合下列例子理解我的上述要求，并且按照要求完成任务。
"""

example1 = """
样例1：
================
输入：CNC组织最早于2019年被发现，其命名来源于所使用的远控木马的PDB路径信息中包含的cnc_client，且该组织主要针对军工和教育行业进行攻击。
输出：['发现','攻击','包含']
================
"""

example2 = """
样例2：
================
输入：攻击者以木尔坦的罗德兰区基于情报的反恐行动(intelligence-based operation，IBO)报告为诱饵，尝试投递一种变种木马程序MessPrint以控制受害者设备。
输出：['使用诱饵','投递','控制']
================
"""

question = """
问题1：
================
输入：{}
输出：
================
"""

prompt_base=background + example1 + example2

In [8]:
pattern = r"\[.*?\]"  # 匹配以 "[" 开始，以 "]" 结束的子串
cnt = -1
result = []
for i in tqdm.tqdm(res):
    prompt = prompt_base + question.format(i)
    response = model.response(prompt)[0]
    matches = re.findall(pattern, response)
    for match in matches:
        try:
            lst = eval(match)  # 使用eval将字符串转化为列表，安全性请自行考虑
            result+=lst
        except:
            pass

        # print(f"========>>>>>>>\nQ:{prompt}\nA:{response}")
        # print(f"========>>>>>>>\nA:{response}")
    if cnt != -1:
        if cnt > 1:
            break
        cnt += 1

    with open("./EventType.json", "w") as file:
        json.dump(result, file)

100%|██████████| 1415/1415 [3:51:44<00:00,  9.83s/it]   
